# 기본 세팅

In [37]:
# 모듈 import
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import time
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, ElementNotInteractableException
from selenium.common.exceptions import ElementClickInterceptedException
import pandas as pd

# 페이지 맨 아래까지 스크롤 다운하는 함수
def scroll_down():
    # 끝까지 스크롤 다운
    last_height = driver.execute_script("return document.body.scrollHeight")

    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # 페이지 로드 대기
        time.sleep(2)  # 페이지 로딩 대기 시간 조정 가능

        # 새로운 높이 계산
        new_height = driver.execute_script("return document.body.scrollHeight")

        # 더 이상 스크롤할 내용이 없으면 종료
        if new_height == last_height:
            break
        last_height = new_height

# 텍스트 찾기 함수
def find_all_text(x):
    if x:
        x_texts = [i.text.strip() for i in x]
        x_text = ', '.join(x_texts)
    else:
        x_text = "Not found"
    
    return x_text

# 메뉴 클릭 함수
def click_menu(menu):
    # 탭 클릭
    try:
    # CSS 선택자를 사용하여 'veBoZ' 클래스를 가진 <span> 요소 중 텍스트가 menu인 요소 찾기
        menu_buttons = driver.find_elements(By.CSS_SELECTOR, 'a._tab-menu span.veBoZ')

        # '메뉴' 텍스트를 가진 요소 클릭
        for button in menu_buttons:
            if menu in button.text:
                button.click()
                break
    except Exception as e:
        print(f"오류 발생: {e}")

# 식당 홈 탭 크롤링
- 식당 이름
- 메뉴 카테고리
- 주소
- 전화번호
- 웹사이트 주소
- 영업 시간
- 역에서부터의 거리
- 서비스 목록
- 총 리뷰 수

In [21]:
def home_page_data():
    time.sleep(5)
    scroll_down()

    # 페이지 소스 가져오기
    page_source = driver.page_source

    # BeautifulSoup 객체 생성
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    
    # 데이터 쌓기
    # 가게 이름
    try:
        restaurant_name = soup.find('span', class_='GHAhO').text.strip()
    except:
        restaurant_name = float('nan')
        
    # 업종
    try:
        category = soup.find('span', class_='lnJFt').text.strip()
    except:
        category = float('nan')
        
    # 총 방문자 리뷰 수
    try:
        count_reviews = soup.find_all('em', class_='place_section_count')
        second_count_review = count_reviews[1].text.strip()
    except:
        second_count_review = float('nan')
        
    # 주소
    try:
        address = soup.find('div', class_='O8qbU tQY7D').find('span', class_='LDgIH').text.strip()
    except:
        address = float('nan')
       
    # 역 기준 거리
    try:
        distance = soup.find('div', class_='nZapA').text.strip()
    except:
        distance = float('nan')

    # 영업 시간
    try:
        business_hours = soup.find('div', class_='O8qbU pSavy').find('span', class_='A_cdD').text.strip()
    except:
        business_hours = float('nan')

    # 전화번호
    try:
        phone_number = soup.find('div', class_='O8qbU nbXkr').find('span', class_='xlx7Q').text.strip()
    except:
        phone_number = float('nan')
        
    # 홈페이지
    try:
        # 'div' 태그에서 class가 'O8qbU yIPfO'인 요소를 찾고, 그 안에 있는 'a' 태그를 모두 찾기
        website_links = soup.find('div', class_='O8qbU yIPfO').find_all('a', class_='place_bluelink')
        # website_links_hrefs = [link.get('href') for link in website_links]
    except:
        website_links = float('nan')
        
    # 부가 서비스 목록
    try:
        home_service = soup.find('div', class_='xPvPE').text.strip()
    except:
        home_service = float('nan')

    return restaurant_name, category, second_count_review, address, distance, business_hours, phone_number, website_links, home_service

In [22]:
home_page_data()

('버거킹 연세로점',
 '햄버거',
 '6,507',
 '서울 서대문구 연세로 25',
 '2신촌역 2번 출구에서 259m미터',
 nan,
 '02-322-3669',
 [<a class="place_bluelink CHmqa" href="http://www.burgerking.co.kr/" role="button">http://www.burgerking.co.kr/</a>],
 '배달')

# 메뉴 탭 크롤링
- 메뉴 텍스트 정보

In [27]:
def menu_page_data():
    #메뉴 탭 클릭
    click_menu('메뉴')
    
    time.sleep(5)
    scroll_down()
    
    # 메뉴 더보기 버튼 클릭
    while True:
        try:
            more_menu_button = driver.find_element(By.XPATH, '/html/body/div[3]/div/div/div/div[6]/div/div[1]/div[2]/div/a')
            if more_menu_button.is_displayed() and more_menu_button.is_enabled():
                more_menu_button.click()
                time.sleep(2)  # 클릭 후 로딩 대기
            else:
                break
        except NoSuchElementException:
            break
    
    # 페이지 소스 가져오기
    page_source = driver.page_source

    # BeautifulSoup 객체 생성
    soup = BeautifulSoup(page_source, 'html.parser')

    # 메뉴 텍스트 내용 긁어오기
    try:
        menu_texts = soup.find_all('div', class_='MXkFw')
        menu_contents = [menu_text.get_text(separator='//').strip() for menu_text in menu_texts]
    except:
        menu_contents = float('nan')
        
    # 결과 반환
    return menu_contents

In [28]:
menu_page_data()

['불맛 더블치즈버거//불맛이 만든 맛의 격차, 더 강렬해진 불맛이 더해진 치즈버거//9,500//원',
 '불맛 더블치즈버거 세트//불맛이 만든 맛의 격차, 더 강렬해진 불맛이 더해진 치즈버거//12,000//원',
 '불맛 더블치즈앤베이컨버거//불맛이 만든 맛의 격차, 더 강렬해진 불맛에 베이컨까지//10,500//원',
 '불맛 더블치즈앤베이컨버거 세트//불맛이 만든 맛의 격차, 더 강렬해진 불맛에 베이컨까지//13,000//원',
 '불맛 더블치즈버거 주니어//불맛이 만든 맛의 격차, 더 강렬해진 불맛이 더해진 치즈버거//5,500//원',
 '불맛 더블치즈팩1//불맛 더블치즈버거+불맛 더블치즈버거 주니어+프렌치프라이R+리얼어니언링+콜라R2//19,900//원',
 '불맛 더블치즈팩2//불맛 더블치즈버거+와퍼+프렌치프라이R+리얼어니언링+콜라R2//21,900//원',
 '불맛 더블치즈팩3//불맛 더블치즈버거+불고기와퍼+프렌치프라이R+리얼어니언링+콜라R2//21,900//원',
 '불맛 더블치즈팩4//불맛 더블치즈버거+불맛 더블치즈앤베이컨버거+프렌치프라이R+리얼어니언링+콜라R2//25,900//원',
 '베이컨치즈와퍼 세트//풍미 가득한 아메리칸 클래식의 완벽한 조화//12,300//원',
 '블랙바비큐콰트로치즈와퍼 세트//블랙번과 특제 BBQ소스, 그리고 불에 직접 구운 100% 순쇠고기 패티, 그리고 네가지 고품격 치즈까지//12,200//원',
 '블랙바비큐와퍼 세트//블랙번과 BBQ소스, 그리고 불에 직접 구운 100% 순쇠고기 패티까지//12,200//원',
 '몬스터와퍼 세트//디아블로 소스의 매콤함, 불에 직접 구운 100% 순쇠고기 패티와 치킨패티까지 압도적 크기의 몬스터와퍼//12,200//원',
 '콰트로치즈와퍼 세트//콰트로치즈와퍼+프렌치프라이R+콜라R//11,300//원',
 '통새우와퍼주니어세트//불맛 가득 순쇠고기, 탱글한 통새우, 매콤한 스파이시토마토소스가 더해진 통새우와퍼주니어//8,700//원',
 '콰트로치즈와퍼주니어세트/

# 정보 탭 크롤링
- 소개글
- 편의시설 및 서비스
- 주차
- 좌석, 공간

In [29]:
def info_page_data():
    # 정보 탭 클릭
    click_menu('정보')

    time.sleep(5)
    scroll_down()

    # 페이지 소스 가져오기
    page_source = driver.page_source

    # BeautifulSoup 객체 생성
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    
    # 데이터 쌓기
    # 소개글
    #     try:
    #         introduce_button = driver.find_element(By.XPATH, '/html/body/div[3]/div/div/div/div[5]/div/div[1]/div/div/div[2]/a')
    #         introduce_button.click()
    #         introduce = soup.find('div', class_='T8RFa').text.strip()
    #     except:
    #         introduce = float('nan')
    
    # 편의 시설 및 서비스
    try:
        service = soup.find_all('div', class_='owG4q')
        services = find_all_text(service)
        
    except:
        services = float('nan')
    
    # 주차
    try:
        parking = soup.find('div', class_='TZ6eS').text.strip()
    except:
        parking = float('nan')
    
    # 좌석, 공간
    try:
        seat = soup.find_all('li', class_='Lw5L1')
        seats = find_all_text(seat)
        
    except:
        seats = float('nan')
        
    return services, parking, seats

In [30]:
info_page_data()

('배달', nan, 'Not found')

# 리뷰 탭 크롤링
- 리뷰 텍스트 내용

In [31]:
# 추가한 코드
from urllib.parse import urlparse, parse_qs, urlencode, urlunparse

def scroll_up():
    driver.execute_script("window.scrollTo(0, 0);")
    time.sleep(2)  # 스크롤 후 페이지 로딩 대기
    
def add_query_param_to_url(url, param_name, param_value):
    """현재 URL에 쿼리 파라미터를 추가하여 반환"""
    url_parts = list(urlparse(url))
    query = dict(parse_qs(url_parts[4]))
    query[param_name] = param_value
    url_parts[4] = urlencode(query, doseq=True)
    return urlunparse(url_parts)

In [ ]:
def review_page_data():
    # 리뷰 탭 클릭
    click_menu('리뷰')
    time.sleep(5)
    
    # 최신순 버튼 클릭
    # 처음에는 이런 코드였는데, scroll_up(), scroll_down() 사용으로 화면에 버튼이 안보이면 최신순 버튼 클릭이 잘 안돼서..ㅠㅠ
    # 또 얘가 되면 키워드 리뷰 더보기 클릭이 안되고.. 이러저러하다가 뭔가 복잡해졌습니다
    # recent_button = driver.find_element(By.XPATH, '/html/body/div[3]/div/div/div/div[6]/div[2]/div[3]/div/div[2]/div[1]/span[2]/a')
    # recent_button.click()
    # time.sleep(3)
    
# scroll_up()
    current_url = driver.current_url
    updated_url = add_query_param_to_url(current_url, 'reviewSort', 'recent')
    driver.get(updated_url)
    time.sleep(5)  # 페이지 로딩 대기
    
    # 키워드 리뷰 데이터 쌓기
    while True:
        try:
            # "더보기" 버튼의 XPath
            more_review_keyword_xpath = '/html/body/div[3]/div/div/div/div[6]/div[2]/div[1]/div/div/div[2]/a[1]'

            # "더보기" 버튼 요소 찾기
            more_review_keyword = driver.find_element(By.XPATH, more_review_keyword_xpath)

            # "더보기" 버튼 클릭
            more_review_keyword.click()

            # 클릭 후 페이지 로딩 대기
            time.sleep(2)

            # 'a' 태그의 'dP0sq' 클래스를 가진 요소 찾기
            dP0sq_elements = driver.find_elements(By.CLASS_NAME, 'dP0sq')

            if not dP0sq_elements:
                # 'dP0sq' 클래스를 가진 요소가 없으면 루프 종료
                break

        except NoSuchElementException:
            # "더보기" 버튼을 찾을 수 없으면 루프 종료
            break

    scroll_down()

    # 리뷰 키워드 정보 추출
    # 페이지 소스 가져오기
    page_source = driver.page_source

    # BeautifulSoup 객체 생성
    soup = BeautifulSoup(page_source, 'html.parser')

    # 모든 't3JSf' 태그와 'CUoLy' 태그를 찾음
    review_keywords_elements = soup.find_all('span', class_='t3JSf')
    review_keywords_count_elements = soup.find_all('span', class_='CUoLy')

    # 두 리스트의 길이를 확인하여 최소 길이만큼 순회
    min_length = min(len(review_keywords_elements), len(review_keywords_count_elements))

    # 키워드 데이터 리스트 생성
    review_keywords_data = []

    # 순서대로 함께 추가
    for i in range(min_length):
        review_keywords = review_keywords_elements[i].text.strip()
        review_keywords_count = review_keywords_count_elements[i].text.strip()
        review_keywords_data.append({
            'keyword': review_keywords,
            'count': review_keywords_count
        })
    
    scroll_down()
    
    # 끝까지 스크롤 다운 및 더보기 클릭 반복
    i = 0
    while i < 10:
        # 끝까지 스크롤 다운
        last_height = driver.execute_script("return document.body.scrollHeight")

        max_scroll_attempts = 10  # 최대 스크롤 시도 횟수 설정
        scroll_attempts = 0

        while True:
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(2)  # 페이지 로딩 대기 시간 조정 가능
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                scroll_attempts += 1
                if scroll_attempts >= max_scroll_attempts:
                    break
            else:
                scroll_attempts = 0  # 높이가 변경되면 시도 횟수 초기화
            last_height = new_height
        
        
        # 더보기 버튼 클릭
        try:
            more_review_button = driver.find_element(By.XPATH, '/html/body/div[3]/div/div/div/div[6]/div[2]/div[3]/div[2]/div/a')
            # 더보기 버튼이 있는 곳으로 스크롤
            driver.execute_script("arguments[0].scrollIntoView(true);", more_review_button)
            time.sleep(1)  # 스크롤 후 로딩 대기
            if more_review_button.is_displayed() and more_review_button.is_enabled():
                more_review_button.click()
                i += 1
                time.sleep(2)  # 클릭 후 로딩 대기
                
        except ElementClickInterceptedException:
            # 스크롤을 조금씩 위로 올리면서 클릭을 시도
            height = -100
            while True:
                driver.execute_script(f"window.scrollBy(0, {height});")
                time.sleep(1)  # 스크롤 후 로딩 대기
                try:
                    more_review_button = driver.find_element(By.XPATH, '/html/body/div[3]/div/div/div/div[6]/div[2]/div[3]/div[2]/div/a')
                    if more_review_button.is_displayed() and more_review_button.is_enabled():
                        more_review_button.click()
                        i += 1
                        time.sleep(2)  # 클릭 후 로딩 대기
                        break  # 클릭 성공 시 루프 탈출
                except ElementClickInterceptedException:
                    height -= 100  # height를 -100씩 더 줄임
                except:
                    break
        except NoSuchElementException:
            break
        except ElementNotInteractableException:
            break
            
            
    # 개별 리뷰 더보기 버튼
    # 초기 XPath
    base_xpath = '/html/body/div[3]/div/div/div/div[6]/div[2]/div[3]/div/ul/li[{}]/div/div[4]/a'

    # 버튼을 순차적으로 클릭
    button_index = 1
    while True:
        try:
            # 현재 XPath
            current_xpath = base_xpath.format(button_index)

            # 해당 XPath에 해당하는 요소가 있는지 확인
            button = driver.find_element(By.XPATH, current_xpath)

            # 요소가 있다면 클릭하고 인덱스를 증가시킴
            button.click()
            time.sleep(2)  # 클릭 후 페이지 로딩 대기
            button_index += 1
        except:
            break
            
    scroll_up()
        
    while True:
        try:
            # CSS 선택자를 사용하여 'sIv5s WPk67' 클래스를 가진 <a> 태그 찾기
            review_buttons = driver.find_elements(By.CSS_SELECTOR, 'a.sIv5s.WPk67[role="button"]')

            if not review_buttons:
                # 더 이상 클릭할 버튼이 없으면 종료
                break

            for button in review_buttons:
                try:
                    button.click()
                    time.sleep(2)  # 클릭 후 페이지 로딩 대기
                except ElementNotInteractableException:
                    continue
        except NoSuchElementException:
            print("리뷰 더보기 버튼을 찾을 수 없습니다.")
            break
        except Exception as e:
            print(f"오류 발생: {e}")
            break
            
    scroll_down()

    # 페이지 로드 후 HTML 소스를 BeautifulSoup로 파싱
    html_source = driver.page_source
    soup = BeautifulSoup(html_source, 'html.parser')
    
    try:
        review_texts = soup.find_all('span', class_='zPfVt')
        keyword_review_texts = soup.find_all('div', class_='ERkm0')
    except:
        review_texts = float('nan')
        keyword_review_texts = float('nan')

    # 텍스트 추출
    review_text_list = [span.get_text() for span in review_texts]
    keyword_review_texts_list = [div.get_text(separator = ',') for div in keyword_review_texts]
    
    return review_keywords_data, review_text_list, keyword_review_texts_list

### 크롤링 실행


In [ ]:
# 데이터프레임 로드
dong = pd.read_excel('./창천동.xlsx')
# dong = pd.read_csv('./창천동.csv', encoding = 'cp949')


# Chrome WebDriver 초기화
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))

# 결과 저장을 위한 빈 리스트 생성
results = []

# 각 ID에 대해 데이터를 수집하고 리스트에 저장
for ID in dong['ID'][5:6]:
    url = f'https://m.place.naver.com/restaurant/{ID}/home?entry=pll'
    driver.get(url)
    
    home_data = home_page_data()
    menu_data = menu_page_data()
    info_data = info_page_data()
    review_data = review_page_data()
    
    # 데이터를 하나의 튜플로 결합
    combined_data = home_data + (menu_data,) + info_data + review_data
    results.append(combined_data)

# 결과를 데이터프레임으로 변환
columns = [
    'restaurant_name', 'category', 'second_count_review', 'address', 'distance', 
    'business_hours', 'phone_number', 'website_links', 'home_service', 'menu_contents', 
    'services', 'parking', 'seats', 'review_keywords_data', 'review_text_list', 'keyword_review_texts_list'
]
results_df = pd.DataFrame(results, columns=columns)

# 기존 데이터프레임과 결합
final_df = pd.concat([dong, results_df], axis=1)

# 데이터프레임 저장
final_df.to_csv('./창천동_상세정보.csv', index=False, encoding='utf-8-sig')


In [62]:
final_df

,Unnamed: 0,지번주소,도로명주소,업체명,업종,URL,ID,restaurant_name,category,second_count_review,...,phone_number,website_links,home_service,menu_contents,services,parking,seats,review_keywords_data,review_text_list,keyword_review_texts_list
0,0,서울특별시 서대문구 창천동 33-36,서울특별시 서대문구 연세로 25 (창천동),버거킹 연세로점,햄버거,https://pcmap.place.naver.com/restaurant/33352...,33352177,한림돈가 본점,돼지고기구이,614,...,0507-1416-6604,[[http://www.neoplate.co.kr]],NaN,"[삼겹살 (180g)//대표//17,000//원, 알목살 (180g)//대표//17...",Not found,NaN,Not found,"[{'keyword': '""음식이 맛있어요""', 'count': '이 키워드를 선택...","[고기 너무 맛있고 서비스도 짱 좋아요:), 고기가 맛있어요. 초벌을 한번 하고 나...","[매장이 넓어요,친절해요,음식이 맛있어요, 음식이 맛있어요,고기 질이 좋아요,잡내가..."
1,1,서울특별시 서대문구 창천동 33-36,서울특별시 서대문구 연세로 25 (창천동),만족커피 신촌연세대점,"카페,디저트",https://pcmap.place.naver.com/restaurant/19226...,1922645907,몽주방 신촌점,요리주점,57,...,0507-1353-2624,NaN,"단체 이용 가능, 예약, 무선 인터넷, 남/녀 화장실 구분, 포장","[통삼겹김치찜//대표//삼겹살 300g, 숙성지 300g,순두부가 같이 통으로 올라...","콜키지 가능, 단체 이용 가능, 예약, 무선 인터넷, 남/녀 화장실 구분, 포장",NaN,"룸최소 6명 ~ 최대 8명, 단체석최소 10명 ~ 최대 30명, 테라스","[{'keyword': '""음식이 맛있어요""', 'count': '이 키워드를 선택...","[압력밥솥에 담겨나오는 닭볶음탕은 처음!!!\n분위기도 좋고 음식도 맛있어요 , 신...","[음식이 맛있어요,인테리어가 멋져요,대화하기 좋아요,단체모임 하기 좋아요, 인테리어..."
2,2,서울특별시 서대문구 창천동 33-36,서울특별시 서대문구 연세로 25 (창천동),카페포엠,카페,https://pcmap.place.naver.com/restaurant/13158...,13158412,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0,"서울특별시 서대문구 창천동 52-5 ,6,7","서울특별시 서대문구 연세로11길 8, 2층 (창천동)",오늘통닭 신촌직영점,"치킨,닭강정",https://pcmap.place.naver.com/restaurant/33040...,33040288,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0,서울특별시 서대문구 창천동 33-55,서울특별시 서대문구 연세로7안길 4 (창천동),한림돈가 본점,돼지고기구이,https://pcmap.place.naver.com/restaurant/37688...,37688284,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,1,서울특별시 서대문구 창천동 33-55,서울특별시 서대문구 연세로7안길 4 (창천동),몽주방 신촌점,요리주점,https://pcmap.place.naver.com/restaurant/19856...,1985698800,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2,서울특별시 서대문구 창천동 33-55,서울특별시 서대문구 연세로7안길 4 (창천동),송준옥,한식,https://pcmap.place.naver.com/restaurant/15227...,1522744322,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,3,서울특별시 서대문구 창천동 33-55,서울특별시 서대문구 연세로7안길 4 (창천동),청춘술방,요리주점,https://pcmap.place.naver.com/restaurant/11896...,1189668345,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,4,서울특별시 서대문구 창천동 33-55,서울특별시 서대문구 연세로7안길 4 (창천동),시장을여는사람들 신촌점,요리주점,https://pcmap.place.naver.com/restaurant/16537...,1653766087,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,5,서울특별시 서대문구 창천동 33-55,서울특별시 서대문구 연세로7안길 4 (창천동),인쌩맥주 신촌점,"맥주,호프",https://pcmap.place.naver.com/restaurant/12273...,1227361695,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [65]:
final_df.review_text_list[0]

['고기 너무 맛있고 서비스도 짱 좋아요:)',
 '고기가 맛있어요. 초벌을 한번 하고 나와서 좋았어요. 직접 고기를 잘라주셔서 편하게 먹을 수 있었습니다. 추천드립니다.',
 '겉바속촉 그 자체에요~육즙도 팡팡터지고 볶음밥까지 순삭했습니다ㅎㅎ맛있게 잘 먹었습니다',
 '고기 질이 좋고 맛잇어용',
 '친구가 추천해줘서 먹으러 왔는데, 너무 만족스러운 식사였습니다!! 덕분에 오늘 하루 좋게 마무리하네요, 감사합니다💗',
 '기본 반찬부터 너무 맛있어서 놀랐습니다! 김치랑만 밥 한 그릇 뚝딱일 듯! 볶음밥 치즈 추가까지 싹 비우고 갑니다 ㅎㅎ ',
 '신촌 돼지고기 맛집하면 한림돈가 먼저 떠올려요. 삼겹살, 가브리살 먹었는데 더 부드러운 건 가브리살이었습니다. 특히 같이 구워 먹는 고사리가 넘 잘 어울리고, 사이드 비빔국수도 새콤하니 고기랑 잘 어울렸습니다. \n여기 오면 꼭 볶음밥은 챙겨 드세요! 뭔가 카레맛이 나는 게 배부른데도 싹싹 긁어 먹게 되는 매력이 있습니다. \n쌈 채소는 한번 시키면 무한 리필이라고 메뉴판에 있어서 주문했는데, 요즘 채소 가격이 올라서 3번까지만 리필 가능하다고 하네요. \n생일맞이 간 거라 간단히 케이크도 했는데 흔쾌히 허락해주신 사장님께 또 감사드립니다. ',
 'ᴍ ᴇ ɴ ᴜ 삼겹살1(180g 16,000₩) + 목살3(180g 16,000₩) + 볶음밥(4000₩) + 치즈추가(2500₩) /\n\n고기 냄새없고 하나도 안기름져서 너무 맛있게 먹고 왔어요!! 불판 위에 야채도 계속 더 가져다 주셨고, 반찬으로 나오는 김치들(그 중에서도 백김치..)과 도토리묵국 너무 잘 어울렸습니다.. 느끼함 싹 보내는 반찬들!!\n고기도 안 질기고 맛있었고, 간만에 가성비에 맛까지 만족스러운 고깃집 찾아서 너무 행복합니다~.~\n\n볶음밥은 계란과 치즈 맛이 많이 나는 슴슴 고소한 맛이였어욥! k후식까지 야무지게 먹고 배부르게 나왔습니다~ 담에 또 갈거에욥',
 '너무 맛있어요~!!^^\n',
 '맛있어여!!']